# Preprocessing

In [21]:
import pandas as pd 
import numpy as np 
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split

import sys
sys.path.append("../src") 
from preprocess import *

In [22]:
train_path = "../data/raw/train.csv"
test_path = "../data/raw/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print(f"Train shape: {train.shape}, Test shape: {test.shape}")
train.head()

Train shape: (140700, 20), Test shape: (93800, 19)


,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


### Train, validation set split

In [25]:
y = train.pop("Depression")
X_train, X_val, y_train, y_val = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state=0)

### Pipeline

In [26]:
drop = ["Name", "City", "Profession", "Academic Pressure", "Work Pressure", "Study Satisfaction", "Job Satisfaction", "Degree"]
oh = ["Gender", "Working Professional or Student", "Sleep Duration", "Dietary Habits", "Have you ever had suicidal thoughts ?", "Family History of Mental Illness" ]

categorical_transformer = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), oh)
    ],
    remainder="passthrough"  # Keeps non-encoded columns
)

preprocessor = Pipeline(steps=[
    ("pressure", FunctionTransformer(assign_pressure)),       # Creates "Pressure"
    ("satisfaction", FunctionTransformer(assign_satisfaction)), # Creates "Satisfaction"
    ("impute", GroupImputer(["Pressure", "Satisfaction", "Financial Stress"])),
    #("profession_fill", FunctionTransformer(fill_profession)),
    ("diet_replace", FunctionTransformer(replace_diet_habits)),
    ("sleep_replace", FunctionTransformer(replace_sleep_duration)),
    ("drop_columns", FunctionTransformer(lambda X: X.drop(columns = drop))),
    ("cat_transformer", categorical_transformer)    # One-hot encoding
])

In [27]:
# Fit the preprocessor on training data
preprocessor.fit(X_train)

# Transform all datasets
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
test_transformed = preprocessor.transform(test)

# Get feature names after one-hot encoding
cat_features = preprocessor.named_steps["cat_transformer"] \
                 .named_transformers_["cat"] \
                 .get_feature_names_out(oh) 

all_features = np.concatenate([
    cat_features,
    [col for col in train.columns if col not in drop and col not in oh],
    ["Pressure", "Satisfaction"]
])
#cat_features = preprocessor.named_transformers_["cat"].get_feature_names_out(oh)
#all_features = np.concatenate([cat_features, preprocessed_train.columns.drop(oh)])

final_train = pd.DataFrame(X_train_transformed, columns=all_features)
final_val = pd.DataFrame(X_val_transformed, columns=all_features)
final_test = pd.DataFrame(test_transformed, columns=all_features)

### Save processed data

In [28]:
processed_train = pd.concat([final_train.reset_index(drop=True), 
                             y_train.reset_index(drop=True).rename("Depression")], axis=1)  
processed_train.to_csv("../data/processed/processed_train.csv", index=False)

processed_val = pd.concat([final_val.reset_index(drop=True), 
                           y_val.reset_index(drop=True).rename("Depression")], axis=1)  
processed_val.to_csv("../data/processed/processed_val.csv", index=False)  

final_test.to_csv("../data/processed/processed_test.csv", index=False)  